# Зависимости



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Веселье с PyLDAVis

In [2]:
! pip install numpy
! pip install pymorphy2[fast]
! pip install nltk


     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 32.8 MB/s 
     |████████████████████████████████| 371 kB 71.1 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=871454 sha256=2bbf2ec81da257339e316cf3a3e391ba21eb257964a109773c66f9cc852c1b04
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [4]:
import random
import nltk
import re
import string
import os
import shutil
from nltk.corpus import stopwords
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from typing import List
import nltk
import string
import pymorphy2
import codecs
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='pymorphy2')

In [6]:
nltk.download('stopwords')
nltk.download('punkt')
stopWordsRu = set(stopwords.words("russian")) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
def get_data() -> pd.DataFrame:
  all_data = pd.read_csv('/content/drive/MyDrive/aiijc_sber/tokens_full_cleaned.csv')
  dop = pd.read_csv('/content/drive/MyDrive/aiijc_sber/full_marked_dataset.csv')
  dop = dop[dop.ans!='неизвестно']
  facts2 = pd.read_csv('/content/drive/MyDrive/aiijc_sber/facts2.csv')
  mit_dict = {0:"животные",
              1: "музыка",
              2: "спорт",
              3: "литература"}
  facts2.labels = [mit_dict[i] for i in facts2.labels]
  data_texts = all_data.texts.to_list() + dop.text.to_list() + facts2.texts.to_list()
  data_ans = [['животные', "музыка", "спорт", "литература"][i] for i in all_data.y] + dop.ans.to_list() + facts2.labels.to_list()
  data = pd.DataFrame()
  data['text'] = data_texts
  data['ans'] = data_ans
  return data


In [9]:
data = get_data()

In [11]:
class Preparator():
  def __init__(self):
      self.morph = pymorphy2.MorphAnalyzer()
      self.tokenizer = nltk.WordPunctTokenizer()
      self.stopwords = set(line.strip() for line in codecs.open('/content/drive/MyDrive/aiijc_sber/all_stopwords.txt', "r", "utf_8_sig").readlines())
      self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")

  def prepare_corp(self, news_list: List[str]):
      return [self.newstext2token(news_text) for news_text in news_list]

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      tokens_bigrammed = self.make_bigrams(tokens_last)
      return tokens_bigrammed

  

In [12]:
prep = Preparator()
all_texts = data.text
print(len(all_texts))
all_tokens = []
for i in range(len(all_texts)):
  all_tokens.append(prep.newstext2token(all_texts[i]))
  if i%1000 == 0:
    print(i)
data['tokens'] = all_tokens

3658
0
1000
2000
3000


In [13]:
def text2vec(tokens: List[str], embeddings: KeyedVectors) -> np.ndarray:
    relevant = 0
    words_vecs = np.zeros((embeddings.vector_size,))
    for word in tokens:
        if word in embeddings:
            words_vecs += embeddings[word]
            relevant += 1

    if relevant:
        words_vecs /= relevant
    return words_vecs

In [14]:
def extract_keywords(tokens: List[str],
                     embeddings: KeyedVectors,
                     top_n: int = 5,
                     diversity: float = 0.5,
                     nr_candidates: int = 20) -> str:
    candidates = []
    doc_embedding = text2vec(tokens, embeddings).reshape(1, -1)
    candidate_embeddings = []
    for candidate in tokens:
        if candidate in embeddings:
            candidates.append(candidate)
            candidate_embeddings.append(embeddings[candidate])

    if len(candidates) == 0:
        return tokens

    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]][::-1]

    return list(set(keywords))

In [15]:
model = KeyedVectors.load("/content/drive/MyDrive/aiijc_sber/keyword_w2v.kv")

all_keywords = []
for tokens in data.tokens:
  # print(tokens)
  tokens = list(filter(lambda x: x.strip()!='', tokens))
  if tokens:
    all_keywords.append(extract_keywords(tokens, model))
  else:
    all_keywords.append([])
data['keywords'] = all_keywords

In [16]:
category2id = ['животные', 'музыка', 'литература', 'спорт']
total_matrix = {}
for doc in range(len(data.tokens)):
  theme = data.ans[doc]
  for word in data.keywords[doc]:
    if word not in total_matrix:
      total_matrix[word] = [0, 0, 0, 0]
    total_matrix[word][category2id.index(theme)] += 1

In [17]:
themes_representation = {'животные': [], 'музыка': [], 'литература': [], 'спорт': []}
for word in total_matrix:
  word_theme = np.array(total_matrix[word]).argmax()
  if total_matrix[word][word_theme]/sum(total_matrix[word]) > 0.5:
    themes_representation[category2id[word_theme]].append((word, (total_matrix[word][word_theme]**2)/sum(total_matrix[word])))

In [18]:
for theme in themes_representation:
  themes_representation[theme] = sorted(themes_representation[theme], key = lambda x: -x[1])

In [19]:
themes_representation['музыка']

[('музыка', 34.02777777777778),
 ('многоголосный', 14.0625),
 ('виолончель', 12.0),
 ('гитара', 12.0),
 ('песня', 11.266666666666667),
 ('октав', 10.0),
 ('такт', 9.0),
 ('слушать', 9.0),
 ('полифония', 8.642857142857142),
 ('старинный', 8.333333333333334),
 ('музыкант', 8.0),
 ('исполнительский', 7.363636363636363),
 ('музыковед', 7.111111111111111),
 ('ритмика', 7.0),
 ('сольфеджио', 7.0),
 ('гвидон', 7.0),
 ('мнемонический', 7.0),
 ('лада', 7.0),
 ('музыкальный', 7.0),
 ('скрипка', 7.0),
 ('музыковедческий', 6.125),
 ('контрапункт', 6.125),
 ('изобрести', 6.125),
 ('записываться', 6.0),
 ('полутон', 6.0),
 ('арфа', 6.0),
 ('композитор', 6.0),
 ('бетховен', 6.0),
 ('звуковысотный', 5.444444444444445),
 ('диез', 5.0),
 ('сонатный', 5.0),
 ('фактура', 5.0),
 ('виртуозный', 5.0),
 ('декламация', 5.0),
 ('многоголосие', 5.0),
 ('струнный', 5.0),
 ('фортепиано', 5.0),
 ('прослушивание', 5.0),
 ('изобретатель', 5.0),
 ('органный', 5.0),
 ('звучание', 5.0),
 ('гобой', 5.0),
 ('чайковский', 

# Nouns only

In [20]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def write_nouns(filename, category):
  with open("/content/drive/MyDrive/aiijc_sber/true_keywords_nouns_" + filename + ".txt", 'w') as dump:
    for word in themes_representation[category]:
      if morph.parse(word[0])[0].tag.POS == "NOUN":
        dump.write(word[0]+'\n')


In [ ]:
write_nouns('music', 'музыка')
write_nouns('literature', 'литература')
write_nouns('sport', 'спорт')
write_nouns('animals', 'животные')

# Actors


In [25]:
def write_nouns_actors(filename, category):
  with open("/content/drive/MyDrive/aiijc_sber/true_keywords_nouns_actors_" + filename + ".txt", 'w') as dump:
    for word in themes_representation[category]:
      if morph.parse(word[0])[0].tag.POS == "NOUN" and morph.parse(word[0])[0].tag.animacy == morph.parse('музыкант')[0].tag.animacy:
        dump.write(word[0]+'\n')


In [26]:
write_nouns_actors('music', 'музыка')
write_nouns_actors('literature', 'литература')
write_nouns_actors('sport', 'спорт')
write_nouns_actors('animals', 'животные')

# Objects

In [27]:
def write_nouns_objects(filename, category):
  with open("/content/drive/MyDrive/aiijc_sber/true_keywords_nouns_objects_" + filename + ".txt", 'w') as dump:
    for word in themes_representation[category]:
      if morph.parse(word[0])[0].tag.POS == "NOUN" and morph.parse(word[0])[0].tag.animacy != morph.parse('музыкант')[0].tag.animacy:
        dump.write(word[0]+'\n')


In [28]:
write_nouns_objects('music', 'музыка')
write_nouns_objects('literature', 'литература')
write_nouns_objects('sport', 'спорт')
write_nouns_objects('animals', 'животные')